In [2]:
# Import necessary libraries
import os
from dotenv import load_dotenv
from web3 import Web3
import json
import time

# Load environment variables
load_dotenv()

# Connect to the Base network
w3 = Web3(Web3.HTTPProvider(os.getenv('RPC_URL')))

# Load contract ABIs
with open('AuctionHouse_ABI.json', 'r') as abi_file:
    auction_house_abi = json.load(abi_file)
    
with open('AuctionHouseFactory_ABI.json', 'r') as abi_file:
    auction_house_factory_abi = json.load(abi_file)
    
with open('AffiliateEscrow_ABI.json', 'r') as abi_file:
    affiliate_escrow_abi = json.load(abi_file)
    
    
with open('AuctionItemERC721_ABI.json', 'r') as abi_file:
    auction_item_erc721_abi = json.load(abi_file)
    
with open('AuctionItemERC721Factory_ABI.json', 'r') as abi_file:
    auction_item_erc721_factory_abi = json.load(abi_file)
    
with open('ERC20_ABI.json', 'r') as abi_file:
    erc20_abi = json.load(abi_file)

# Set up contract addresses
AUCTION_HOUSE_FACTORY_ADDRESS = os.getenv('AUCTION_HOUSE_FACTORY_ADDRESS')
AUCTION_ITEM_ERC721_FACTORY_ADDRESS = os.getenv('AUCTION_ITEM_ERC721_FACTORY_ADDRESS')

# Set up contract instances
auction_house_factory = w3.eth.contract(
    address=AUCTION_HOUSE_FACTORY_ADDRESS, 
    abi=auction_house_factory_abi
)


auction_item_erc721_factory = w3.eth.contract(
    address=AUCTION_ITEM_ERC721_FACTORY_ADDRESS, 
    abi=auction_item_erc721_factory_abi
)

# Set up accounts
owner_account = w3.eth.account.from_key(os.getenv('OWNER_PRIVATE_KEY'))
seller_account = w3.eth.account.from_key(os.getenv('SELLER_PRIVATE_KEY'))
bidder1_account = w3.eth.account.from_key(os.getenv('BIDDER1_PRIVATE_KEY'))
bidder2_account = w3.eth.account.from_key(os.getenv('BIDDER2_PRIVATE_KEY'))
affiliate_account = w3.eth.account.from_key(os.getenv('AFFILIATE_PRIVATE_KEY'))
arbiter_account = w3.eth.account.from_key(os.getenv('ARBITER_PRIVATE_KEY'))

# Example encrypted message structure for bids
EXAMPLE_MESSAGE = {
    "encryptedData": "0x",
    "ephemeralPublicKey": "0x",
    "iv": "0x",
    "verificationHash": "0x"
}

# Constants for the auction system
DEFAULT_SETTLEMENT_DEADLINE = 21 * 24 * 60 * 60  # 21 days in seconds

In [3]:
# Helper functions for common operations

def create_auction_house(name, image, description, custom_deadline=0):
    """Create a new auction house using the factory"""
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = auction_house_factory.functions.createAuctionHouse(
        name,
        image,
        description,
        custom_deadline
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Transaction receipt: {tx_receipt}")
    
    # Process logs to get the new auction house address
    for log in tx_receipt['logs']:
        try:
            event = auction_house_factory.events.AuctionHouseCreated().process_log(log)
            if event:
                return event['args']['auctionHouse']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionHouseCreated event found in the transaction logs")
    return None

def create_auction_item_erc721(name, symbol, contract_uri):
    """Create a new AuctionItemERC721 contract using the factory"""
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = auction_item_erc721_factory.functions.createAuctionItemERC721(
        name,
        symbol,
        contract_uri
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Process logs to get the new token contract address
    for log in tx_receipt['logs']:
        try:
            event = auction_item_erc721_factory.events.AuctionItemERC721Created().process_log(log)
            if event:
                token_address = event['args']['tokenContract']
                print(f"Created new AuctionItemERC721 contract at: {token_address}")
                return token_address
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionItemERC721Created event found in the transaction logs")
    return None

def mint_auction_item(token_address, metadata, to_address=None):
    """Mint a new token with metadata and return its ID"""
    # Create contract instance
    token_contract = w3.eth.contract(
        address=token_address,
        abi=auction_item_erc721_abi
    )
    
    # Use provided address or default to owner account
    mint_to_address = to_address if to_address else owner_account.address
    
    # Mint token to the specified address
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = token_contract.functions.mintWithMetadata(
        mint_to_address,
        metadata["name"],
        metadata["description"],
        metadata["image"],
        metadata["termsOfService"],
        metadata.get("supplementalImages", [])
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Get the token ID from the event
    token_id = None
    for log in tx_receipt['logs']:
        try:
            event = token_contract.events.Transfer().process_log(log)
            if event and event['args']['from'] == '0x0000000000000000000000000000000000000000':
                token_id = event['args']['tokenId']
                print(f"Minted token ID: {token_id} to address: {mint_to_address}")
                break
        except Exception as e:
            continue
    
    if token_id is None:
        print("Failed to get token ID from mint transaction")
        return None
    
    return token_id

def prepare_nft_for_auction(auction_house_address, name, symbol, contract_uri, metadata):
    """Create an NFT, set metadata, and transfer to auction house in one function"""
    # Step 1: Create a new NFT contract
    token_address = create_auction_item_erc721(name, symbol, contract_uri)
    if not token_address:
        print("Failed to create NFT contract")
        return None, None
    
    # Step 2: Mint a token with metadata
    token_id = mint_auction_item(token_address, metadata)
    if not token_id:
        print("Failed to mint token")
        return token_address, None
    

    
    print(f"NFT prepared for auction: Contract {token_address}, Token ID {token_id}")
    return token_address, token_id


def create_auction(auction_house_contract, token_address, token_id, start_time, reserve_price, 
                  duration, affiliate_fee, arbiter_address, escrow_factory_address, 
                  payment_token=None, premium=False, premium_rate=0, 
                  min_bid_increment=500, time_extension=900):
    """Create a new auction"""
    if payment_token is None:
        payment_token = "0x0000000000000000000000000000000000000000"  # ETH
        
    nonce = w3.eth.get_transaction_count(seller_account.address)
    
    txn = auction_house_contract.functions.createAuction(
        token_address,
        token_id,
        start_time,
        reserve_price,
        duration,
        affiliate_fee,
        arbiter_address,
        escrow_factory_address,
        payment_token,
        premium,
        premium_rate,
        min_bid_increment,
        time_extension
    ).build_transaction({
        'from': seller_account.address,
        'nonce': nonce,
        'gas': 5000000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = seller_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Process logs to get the auction ID
    for log in tx_receipt['logs']:
        try:
            event = auction_house_contract.events.AuctionCreated().process_log(log)
            if event:
                return event['args']['auctionId']
        except Exception as e:
            print(f"Error processing log: {e}")
            
    print("No AuctionCreated event found in the transaction logs")
    return None

def place_bid(auction_house_contract, auction_id, bidder_account, affiliate_address, message, bid_amount):
    """Place a bid on an auction"""
    nonce = w3.eth.get_transaction_count(bidder_account.address)
    
    txn = auction_house_contract.functions.createBid(
        auction_id,
        affiliate_address,
        message,
        bid_amount
    ).build_transaction({
        'from': bidder_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
        'value': bid_amount  # For ETH bids
    })
    
    signed_txn = bidder_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Placed bid of {bid_amount} wei on auction {auction_id}")
    return tx_receipt

def end_auction(auction_house_contract, auction_id):
    """End an auction"""
    nonce = w3.eth.get_transaction_count(owner_account.address)
    
    txn = auction_house_contract.functions.endAuction(
        auction_id
    ).build_transaction({
        'from': owner_account.address,
        'nonce': nonce,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
    })
    
    signed_txn = owner_account.sign_transaction(txn)
    tx_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print(f"Ended auction {auction_id}")
    return tx_receipt

def get_minimum_bid(auction_house_contract, auction_id):
    """Get the minimum bid required for an auction"""
    min_bid = auction_house_contract.functions.getMinimumBid(auction_id).call()
    print(f"Minimum bid required: {w3.from_wei(min_bid, 'ether')} ETH")
    return min_bid



In [4]:
# Step 1: Create a new auction house
auction_house_address = create_auction_house(
    "Test Auction House",
    "https://example.com/image.png",
    "A test auction house for NFTs",
    30 * 24 * 60 * 60  # 30 days settlement deadline
)

print(f"New auction house created at: {auction_house_address}")

# Create auction house contract instance
auction_house = w3.eth.contract(
    address=auction_house_address,
    abi=auction_house_abi
)

Transaction receipt: AttributeDict({'blockHash': HexBytes('0x50b9c84c4555d4d787afa341809c0e76338abc6d944f5072c110c9b57d9e4505'), 'blockNumber': 29503606, 'contractAddress': None, 'cumulativeGasUsed': 45512647, 'effectiveGasPrice': 2479306, 'from': '0x9f4640d04371ff6b7886ade5323746388107723a', 'gasUsed': 2707175, 'l1BaseFeeScalar': '0x8dd', 'l1BlobBaseFee': '0xe09386b', 'l1BlobBaseFeeScalar': '0x101c12', 'l1Fee': '0x98d4d8a7f', 'l1GasPrice': '0x13ea86cf', 'l1GasUsed': '0x9d5', 'logs': [AttributeDict({'address': '0x991cEf5027997d248dAA544FC2434E45Dd98A199', 'topics': [HexBytes('0xd572739fdc5a0a4fcfa09f49f2af7f1df24c6506fa3de300a41e524c93fcdd93'), HexBytes('0x00000000000000000000000091929caa0015e9cce3e487c9249f3cf530c70904'), HexBytes('0x0000000000000000000000009f4640d04371ff6b7886ade5323746388107723a')], 'data': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000e000000000000000000000000000000000000000

In [6]:
# Step 2: Create an NFT contract for multiple auctions

x= create_auction_item_erc721(
    "Test NFT",
    "TEST",
    "https://example.com/nft.json"
)


Error processing log: The event signature did not match the provided ABI
Error processing log: The event signature did not match the provided ABI
Error processing log: 'tokenContract'
No AuctionItemERC721Created event found in the transaction logs


In [7]:
x